In [2]:
import tensorflow as tf
import torch
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import os

In [2]:
train_data = pd.read_csv('data/final/relabel/final_train_data.csv')
train_data = train_data[['Review', 'Relabel']]
train_data.loc[(train_data['Relabel'] == "부정"), 'Relabel'] = 0
train_data.loc[(train_data['Relabel'] == "긍정"), 'Relabel'] = 1

In [3]:
train_data

,Review,Relabel
0,그냥 딱 정핏 이에요 원하던 핏 나와서 너무 좋아요,1
1,모카베이지라 그냥 베이지보다 묵직한 느낌이에요,1
2,저번에 제주니트 샀을 때 핏이 너무 좋아서 검은색도 샀는데 엘무드 역시 좋네요~~~,1
3,올해 제품이 별로다 ㄴㄴㄴ작년 거가 오진다 ooo173에 하비인데요 작년 거 살 때...,0
4,"막 별로는 아닌데 기대 이하였습니다. 딱 3점정도워낙 자자해서 구매했는데, 재질 그...",0
...,...,...
10889,밑위가 좀 작은 거 빼곤 전반적으로 만족합니다. 괜찮네요,0
10890,이뻐요 엄청 오버핏이라 팔통이 커요 기장은 짧은데 통은 엄청 널벙요,0
10891,어깨가 넓어보여요. 색은 사진과 같고 부드럽습니다.,1
10892,올해 특히 더 발마칸 유행이라 주문이 많은지 배송이 조금 늦게 왔지만 옷을 받아보고...,1


In [4]:
train_data['Review'] = train_data['Review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
print(train_data.isnull().sum())

Review     0
Relabel    0
dtype: int64


/var/folders/8s/_473l8ss1hx20zy1txdh03x40000gn/T/ipykernel_78631/3320672272.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['Review'] = train_data['Review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [6]:
train_data.to_csv('train_data.txt', sep = '\t', index = False)

In [4]:
test_data = pd.read_csv('data/final/test_data.csv')
test_data

,Type,Rate,Review
0,화이트 멜란지/XL,4,178/76 기준으로 L 사이즈는 좀 작아서 XL로 다시 시켰더니 딱 맞습니다. 두...
1,오트밀/XL,5,생각보다 작습니다. 제 스펙이면 무조건 XL 가셔야 하고 팔 길이도 딱 손목시계 차...
2,화이트 멜란지/L,5,색감은 연한 그레이색감인데 화이트가 군데군데 들어가서 좀더 밝은색감이 예쁘네요. 핏...
3,오트밀/M,5,좋은 제품이네요. 여자친구가 잘 샀다고 칭찬해줬어요.일단 옷감도 괜찮고 두툼하니 겨...
4,오트밀/M,5,자인도 이쁘고 따뜻해서 겨울에 입기 좋을거 같아영
...,...,...,...
11986,퓨어 블랙/L,5,사이즈 너무 잘 맞고 이뻐요! 아직 낮에는 좀 덥지만 선선해지면 잘 입을꺼같아요 !...
11987,퓨어 블랙/M,5,아직 입기는 조금 더운 날씨이지만 아침 저녁으로 입으면 따뜻하고 좋아요
11988,콘크리트 그레이/L,5,이가격에 하프집업니트 좋아요 근데 세탁기에 돌려도 되는지 모르겠네요
11989,콘크리트 그레이/L,5,어디에나 잘 어울리는 상의입니다.재질도 독톡하고 아주 좋습니다.품도 넉넉해서 요즘 ...


In [6]:
test_data = test_data[['Review']]
test_data

,Review
0,178/76 기준으로 L 사이즈는 좀 작아서 XL로 다시 시켰더니 딱 맞습니다. 두...
1,생각보다 작습니다. 제 스펙이면 무조건 XL 가셔야 하고 팔 길이도 딱 손목시계 차...
2,색감은 연한 그레이색감인데 화이트가 군데군데 들어가서 좀더 밝은색감이 예쁘네요. 핏...
3,좋은 제품이네요. 여자친구가 잘 샀다고 칭찬해줬어요.일단 옷감도 괜찮고 두툼하니 겨...
4,자인도 이쁘고 따뜻해서 겨울에 입기 좋을거 같아영
...,...
11986,사이즈 너무 잘 맞고 이뻐요! 아직 낮에는 좀 덥지만 선선해지면 잘 입을꺼같아요 !...
11987,아직 입기는 조금 더운 날씨이지만 아침 저녁으로 입으면 따뜻하고 좋아요
11988,이가격에 하프집업니트 좋아요 근데 세탁기에 돌려도 되는지 모르겠네요
11989,어디에나 잘 어울리는 상의입니다.재질도 독톡하고 아주 좋습니다.품도 넉넉해서 요즘 ...


In [7]:
test_data['Review'] = test_data['Review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
print(test_data.isnull().sum())

Review    0
dtype: int64


/var/folders/8s/_473l8ss1hx20zy1txdh03x40000gn/T/ipykernel_7499/1508574037.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['Review'] = test_data['Review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [9]:
test_data

,Review
0,기준으로 사이즈는 좀 작아서 로 다시 시켰더니 딱 맞습니다 두께는 두껍지도 않고...
1,생각보다 작습니다 제 스펙이면 무조건 가셔야 하고 팔 길이도 딱 손목시계 차는 곳...
2,색감은 연한 그레이색감인데 화이트가 군데군데 들어가서 좀더 밝은색감이 예쁘네요 핏감...
3,좋은 제품이네요 여자친구가 잘 샀다고 칭찬해줬어요일단 옷감도 괜찮고 두툼하니 겨울에...
4,자인도 이쁘고 따뜻해서 겨울에 입기 좋을거 같아영
...,...
11986,사이즈 너무 잘 맞고 이뻐요 아직 낮에는 좀 덥지만 선선해지면 잘 입을꺼같아요 제...
11987,아직 입기는 조금 더운 날씨이지만 아침 저녁으로 입으면 따뜻하고 좋아요
11988,이가격에 하프집업니트 좋아요 근데 세탁기에 돌려도 되는지 모르겠네요
11989,어디에나 잘 어울리는 상의입니다재질도 독톡하고 아주 좋습니다품도 넉넉해서 요즘 입기...


In [8]:
test_data.to_csv('test_data_상의.txt', sep = '\t', index = False)